# Generative Adversarial Network

ref:

* obs: use Python 3

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten
from keras.layers import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, Model
from keras.optimizers import Adam

Using TensorFlow backend.


## Generator network

In [3]:
def build_generator(latent_dim, img_shape):

    model = Sequential()

    model.add(Dense(256, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(1024))
    model.add(LeakyReLU(alpha=0.2))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(np.prod(img_shape), activation='tanh'))
    model.add(Reshape(img_shape))

    model.summary()

    noise = Input(shape=(latent_dim,))
    img = model(noise)

    return Model(noise, img)

# Discriminator network

In [4]:
def build_discriminator(img_shape):

    model = Sequential()

    model.add(Flatten(input_shape=img_shape))
    model.add(Dense(512))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(256))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()

    img = Input(shape=img_shape)
    validity = model(img)

    return Model(img, validity)

In [5]:
# sample images from generator
def sample_images(epoch, latent_dim, G):
    r, c = 5, 5
    noise = np.random.normal(0, 1, (r * c, latent_dim))
    gen_imgs = G.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
            axs[i,j].axis('off')
            cnt += 1
    fig.savefig("images/%d.png" % epoch)
    plt.close()

# train the GAN

In [6]:
def train(epochs, D, G, combined, latent_dim, batch_size=128, sample_interval=50):

    # Load the dataset
    (X_train, _), (_, _) = mnist.load_data()

    # Rescale -1 to 1
    X_train = X_train / 127.5 - 1.
    X_train = np.expand_dims(X_train, axis=3)

    # Adversarial ground truths
    valid = np.ones((batch_size, 1))
    fake = np.zeros((batch_size, 1))

    for epoch in range(epochs):

        # ---------------------
        #  Train Discriminator
        # ---------------------

        # Select a random batch of images
        idx = np.random.randint(0, X_train.shape[0], batch_size)
        imgs = X_train[idx]

        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Generate a batch of new images
        gen_imgs = G.predict(noise)

        # Train the discriminator
        d_loss_real = D.train_on_batch(imgs, valid)
        d_loss_fake = D.train_on_batch(gen_imgs, fake)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # ---------------------
        #  Train Generator
        # ---------------------

        noise = np.random.normal(0, 1, (batch_size, latent_dim))

        # Train the generator (to have the discriminator label samples as valid)
        g_loss = combined.train_on_batch(noise, valid)

        # Plot the progress
        print ("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[1], g_loss))

        # If at save interval => save generated image samples
        if epoch % sample_interval == 0:
            sample_images(epoch, latent_dim, G)

## main()

In [7]:
img_rows = 28
img_cols = 28
channels = 1
img_shape = (img_rows, img_cols, channels)

In [8]:
latent_dim = 100

In [9]:
# optimizer

In [10]:
optimizer = Adam(0.0002, 0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [11]:
# Build and compile the discriminator

In [12]:
D = build_discriminator(img_shape)
D.compile(loss='binary_crossentropy',
          optimizer=optimizer,
          metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               401920    
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 533,505
Trainable params: 533,505
Non-trainable params: 0
________________________________________________

In [13]:
# Build the generator

In [14]:
G = build_generator(latent_dim, img_shape)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 256)               25856     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 256)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 256)               1024      
_________________________________________________________________
dense_5 (Dense)              (None, 512)               131584    
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 512)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dense_6 (Dense)              (None, 1024)             

In [15]:
# The generator takes noise as input and generates imgs

In [16]:
z = Input(shape=(latent_dim,))
img = G(z)

In [17]:
# For the combined model we will only train the generator
D.trainable = False

In [18]:
# The discriminator takes generated images as input and determines validity
validity = D(img)

In [19]:
# The combined model  (stacked generator and discriminator)
# Trains the generator to fool the discriminator

In [20]:
combined = Model(z, validity)
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

In [21]:
# epochs = 30000
epochs = 10000
train(epochs=epochs, D=D, G=G, combined=combined, latent_dim=latent_dim, batch_size=32, sample_interval=200)

c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [D loss: 0.542124, acc.: 57.81%] [G loss: 0.569858]


c:\users\henri\appdata\local\programs\python\python37\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


1 [D loss: 0.358455, acc.: 75.00%] [G loss: 0.598678]
2 [D loss: 0.341962, acc.: 73.44%] [G loss: 0.699100]
3 [D loss: 0.315618, acc.: 92.19%] [G loss: 0.803947]
4 [D loss: 0.280758, acc.: 93.75%] [G loss: 1.012944]
5 [D loss: 0.255767, acc.: 96.88%] [G loss: 1.145375]
6 [D loss: 0.219745, acc.: 98.44%] [G loss: 1.321876]
7 [D loss: 0.172782, acc.: 100.00%] [G loss: 1.529398]
8 [D loss: 0.140267, acc.: 100.00%] [G loss: 1.745231]
9 [D loss: 0.116834, acc.: 100.00%] [G loss: 1.971421]
10 [D loss: 0.104446, acc.: 100.00%] [G loss: 2.077282]
11 [D loss: 0.073167, acc.: 100.00%] [G loss: 2.216881]
12 [D loss: 0.088353, acc.: 100.00%] [G loss: 2.368895]
13 [D loss: 0.068108, acc.: 100.00%] [G loss: 2.543435]
14 [D loss: 0.058572, acc.: 100.00%] [G loss: 2.589886]
15 [D loss: 0.045702, acc.: 100.00%] [G loss: 2.815343]
16 [D loss: 0.039979, acc.: 100.00%] [G loss: 2.782859]
17 [D loss: 0.032444, acc.: 100.00%] [G loss: 2.984042]
18 [D loss: 0.042038, acc.: 100.00%] [G loss: 2.868037]
19 [D l

150 [D loss: 0.080343, acc.: 98.44%] [G loss: 4.563143]
151 [D loss: 0.134394, acc.: 96.88%] [G loss: 3.876487]
152 [D loss: 0.109533, acc.: 96.88%] [G loss: 4.876696]
153 [D loss: 0.299121, acc.: 87.50%] [G loss: 3.129858]
154 [D loss: 0.116754, acc.: 95.31%] [G loss: 3.993886]
155 [D loss: 0.113430, acc.: 96.88%] [G loss: 4.809875]
156 [D loss: 0.210385, acc.: 92.19%] [G loss: 4.078090]
157 [D loss: 0.068797, acc.: 98.44%] [G loss: 4.613706]
158 [D loss: 0.082676, acc.: 100.00%] [G loss: 4.332349]
159 [D loss: 0.128984, acc.: 95.31%] [G loss: 5.019379]
160 [D loss: 0.113356, acc.: 98.44%] [G loss: 4.235505]
161 [D loss: 0.087667, acc.: 95.31%] [G loss: 5.110422]
162 [D loss: 0.338944, acc.: 84.38%] [G loss: 3.486944]
163 [D loss: 0.094215, acc.: 96.88%] [G loss: 4.511008]
164 [D loss: 0.220442, acc.: 87.50%] [G loss: 3.489964]
165 [D loss: 0.115714, acc.: 93.75%] [G loss: 4.622981]
166 [D loss: 0.860339, acc.: 64.06%] [G loss: 2.437475]
167 [D loss: 0.524533, acc.: 76.56%] [G loss: 3

...

9339 [D loss: 0.634197, acc.: 62.50%] [G loss: 1.010992]
9340 [D loss: 0.662010, acc.: 56.25%] [G loss: 0.943570]
9341 [D loss: 0.677384, acc.: 62.50%] [G loss: 0.984597]
9342 [D loss: 0.594372, acc.: 67.19%] [G loss: 1.077479]
9343 [D loss: 0.635540, acc.: 60.94%] [G loss: 0.941255]
9344 [D loss: 0.614242, acc.: 65.62%] [G loss: 1.026507]
9345 [D loss: 0.646580, acc.: 56.25%] [G loss: 1.004177]
9346 [D loss: 0.649643, acc.: 62.50%] [G loss: 0.970091]
9347 [D loss: 0.656932, acc.: 67.19%] [G loss: 0.987722]
9348 [D loss: 0.629595, acc.: 65.62%] [G loss: 0.979536]
9349 [D loss: 0.600919, acc.: 62.50%] [G loss: 0.956341]
9350 [D loss: 0.700433, acc.: 59.38%] [G loss: 0.915447]
9351 [D loss: 0.616694, acc.: 57.81%] [G loss: 0.915618]
9352 [D loss: 0.588925, acc.: 62.50%] [G loss: 0.877694]
9353 [D loss: 0.583606, acc.: 70.31%] [G loss: 0.956538]
9354 [D loss: 0.638175, acc.: 67.19%] [G loss: 0.953776]
9355 [D loss: 0.656010, acc.: 56.25%] [G loss: 0.945241]
9356 [D loss: 0.587093, acc.: 7

9482 [D loss: 0.660282, acc.: 54.69%] [G loss: 1.079129]
9483 [D loss: 0.617618, acc.: 60.94%] [G loss: 0.888118]
9484 [D loss: 0.641729, acc.: 65.62%] [G loss: 0.885563]
9485 [D loss: 0.642006, acc.: 65.62%] [G loss: 0.945399]
9486 [D loss: 0.639678, acc.: 56.25%] [G loss: 0.986097]
9487 [D loss: 0.576949, acc.: 73.44%] [G loss: 0.978547]
9488 [D loss: 0.660445, acc.: 62.50%] [G loss: 1.013880]
9489 [D loss: 0.651628, acc.: 59.38%] [G loss: 0.982031]
9490 [D loss: 0.639739, acc.: 56.25%] [G loss: 1.092963]
9491 [D loss: 0.687735, acc.: 53.12%] [G loss: 0.994855]
9492 [D loss: 0.623186, acc.: 67.19%] [G loss: 0.956668]
9493 [D loss: 0.599763, acc.: 70.31%] [G loss: 0.938079]
9494 [D loss: 0.639671, acc.: 57.81%] [G loss: 0.932903]
9495 [D loss: 0.588222, acc.: 71.88%] [G loss: 0.930387]
9496 [D loss: 0.626246, acc.: 62.50%] [G loss: 0.963476]
9497 [D loss: 0.621506, acc.: 56.25%] [G loss: 0.969429]
9498 [D loss: 0.587834, acc.: 67.19%] [G loss: 0.988023]
9499 [D loss: 0.629633, acc.: 6

9627 [D loss: 0.662089, acc.: 56.25%] [G loss: 0.929691]
9628 [D loss: 0.629268, acc.: 57.81%] [G loss: 0.925559]
9629 [D loss: 0.636631, acc.: 65.62%] [G loss: 0.932222]
9630 [D loss: 0.639202, acc.: 62.50%] [G loss: 0.930117]
9631 [D loss: 0.608269, acc.: 64.06%] [G loss: 1.003902]
9632 [D loss: 0.654682, acc.: 62.50%] [G loss: 0.955265]
9633 [D loss: 0.592372, acc.: 73.44%] [G loss: 0.954799]
9634 [D loss: 0.543409, acc.: 71.88%] [G loss: 0.975837]
9635 [D loss: 0.701910, acc.: 59.38%] [G loss: 0.914044]
9636 [D loss: 0.600634, acc.: 64.06%] [G loss: 0.955063]
9637 [D loss: 0.622250, acc.: 64.06%] [G loss: 0.944582]
9638 [D loss: 0.669299, acc.: 68.75%] [G loss: 0.922886]
9639 [D loss: 0.637670, acc.: 65.62%] [G loss: 1.020528]
9640 [D loss: 0.625655, acc.: 59.38%] [G loss: 1.012579]
9641 [D loss: 0.631272, acc.: 64.06%] [G loss: 1.066399]
9642 [D loss: 0.685425, acc.: 60.94%] [G loss: 1.002233]
9643 [D loss: 0.555146, acc.: 73.44%] [G loss: 1.025793]
9644 [D loss: 0.615915, acc.: 6

9772 [D loss: 0.600806, acc.: 67.19%] [G loss: 0.838597]
9773 [D loss: 0.691510, acc.: 59.38%] [G loss: 0.998600]
9774 [D loss: 0.669816, acc.: 54.69%] [G loss: 1.035767]
9775 [D loss: 0.627154, acc.: 60.94%] [G loss: 0.878025]
9776 [D loss: 0.653947, acc.: 57.81%] [G loss: 0.985373]
9777 [D loss: 0.619760, acc.: 65.62%] [G loss: 0.989791]
9778 [D loss: 0.627829, acc.: 68.75%] [G loss: 0.962546]
9779 [D loss: 0.591013, acc.: 73.44%] [G loss: 1.017445]
9780 [D loss: 0.565556, acc.: 78.12%] [G loss: 1.006816]
9781 [D loss: 0.572701, acc.: 68.75%] [G loss: 1.081617]
9782 [D loss: 0.659756, acc.: 57.81%] [G loss: 0.973458]
9783 [D loss: 0.692970, acc.: 57.81%] [G loss: 0.950661]
9784 [D loss: 0.670760, acc.: 60.94%] [G loss: 0.920383]
9785 [D loss: 0.658145, acc.: 59.38%] [G loss: 0.866746]
9786 [D loss: 0.647230, acc.: 60.94%] [G loss: 0.974667]
9787 [D loss: 0.608197, acc.: 60.94%] [G loss: 1.064486]
9788 [D loss: 0.675836, acc.: 57.81%] [G loss: 1.045123]
9789 [D loss: 0.669706, acc.: 6

9916 [D loss: 0.631178, acc.: 67.19%] [G loss: 1.012461]
9917 [D loss: 0.634446, acc.: 62.50%] [G loss: 0.943445]
9918 [D loss: 0.657357, acc.: 64.06%] [G loss: 0.882001]
9919 [D loss: 0.688313, acc.: 62.50%] [G loss: 0.982514]
9920 [D loss: 0.616416, acc.: 68.75%] [G loss: 0.866336]
9921 [D loss: 0.574692, acc.: 67.19%] [G loss: 0.995735]
9922 [D loss: 0.481138, acc.: 81.25%] [G loss: 0.913948]
9923 [D loss: 0.655128, acc.: 62.50%] [G loss: 0.973752]
9924 [D loss: 0.660664, acc.: 64.06%] [G loss: 0.937541]
9925 [D loss: 0.660419, acc.: 57.81%] [G loss: 0.862113]
9926 [D loss: 0.645624, acc.: 56.25%] [G loss: 0.921814]
9927 [D loss: 0.629637, acc.: 65.62%] [G loss: 0.974813]
9928 [D loss: 0.611024, acc.: 65.62%] [G loss: 0.861597]
9929 [D loss: 0.647226, acc.: 60.94%] [G loss: 0.988414]
9930 [D loss: 0.674483, acc.: 59.38%] [G loss: 1.260759]
9931 [D loss: 0.646582, acc.: 70.31%] [G loss: 1.067206]
9932 [D loss: 0.635675, acc.: 56.25%] [G loss: 1.075556]
9933 [D loss: 0.698812, acc.: 5